In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [3]:
paths = {
    "ssd": {
        "split_1": "./results/ssd/training/split_1/events.out.tfevents.1748736001.ip-10-192-12-217.79847.0",
        "split_2": "./results/ssd/training/split_2/events.out.tfevents.1748736972.ip-10-192-12-217.79847.1",
        "split_3": "./results/ssd/training/split_3/events.out.tfevents.1748737976.ip-10-192-12-217.79847.2",
        "split_4": "./results/ssd/training/split_4/events.out.tfevents.1748739021.ip-10-192-12-217.79847.3"
    },
    "yolo": {
        "split_1": "./results/yolo/training/split_1/events.out.tfevents.1748475122.ip-10-192-11-39.8503.0",
        "split_2": "./results/yolo/training/split_2/events.out.tfevents.1748475854.ip-10-192-11-39.8503.1",
        "split_3": "./results/yolo/training/split_3/events.out.tfevents.1748476565.ip-10-192-11-39.8503.2",
        "split_4": "./results/yolo/training/split_4/events.out.tfevents.1748477314.ip-10-192-11-39.8503.3"
    }
}

In [4]:
data = {
    "ssd": {
        "split_1": EventAccumulator(paths["ssd"]["split_1"]).Reload(),
        "split_2": EventAccumulator(paths["ssd"]["split_2"]).Reload(),
        "split_3": EventAccumulator(paths["ssd"]["split_3"]).Reload(),
        "split_4": EventAccumulator(paths["ssd"]["split_4"]).Reload()
    },
    "yolo": {
        "split_1": EventAccumulator(paths["yolo"]["split_1"]).Reload(),
        "split_2": EventAccumulator(paths["yolo"]["split_2"]).Reload(),
        "split_3": EventAccumulator(paths["yolo"]["split_3"]).Reload(),
        "split_4": EventAccumulator(paths["yolo"]["split_4"]).Reload()
    }
}

In [5]:
ssd_tags = data['ssd']['split_1'].Tags()['scalars']
ssd_tags

['Loss/train',
 'mAP/val',
 'mAP/val_50',
 'Precision/val',
 'Recall/val',
 'IoU/val']

In [6]:
custom_titles = {
    'mAP/val': "SSD mAP",
    'Precision/val': "SSD Precision",
    'Recall/val': "SSD Recall",
    'IoU/val': "SSD IoU",
}

for ssd_tag in ssd_tags:
    
    df = pd.DataFrame()
    #set a column for each split
    for split in data['ssd']:
        split_data_complete_run = []
        split_data = data['ssd'][split].Scalars(ssd_tag)
        for event_data in split_data:
            #split_data_complete_run[event_data.step] = event_data.value
            split_data_complete_run.append((event_data.step, event_data.value))
        
        # convert to dataframe
        split_data_complete_run = pd.DataFrame(split_data_complete_run, columns=['step', split])
        #set index to step
        split_data_complete_run.set_index('step', inplace=True)
        #merge with df
        if df.empty:
            df = split_data_complete_run
        else:
            df = df.join(split_data_complete_run, how='outer')

    #plot the data
    plt.figure(figsize=(10, 6)) 
    sns.lineplot(
        data=df,
        dashes=False,
        alpha=0.5
    )

    
    # create a mean column, it is the mean of all the splits
    mean = df.mean(axis=1)
    sns.lineplot(
        data=mean,
        label="Mean",
        linewidth=2,
        color="red"
    )

    plt.title(custom_titles[ssd_tag] if ssd_tag in custom_titles else ssd_tag)
    plt.xlabel('Step')
    plt.ylabel('Value')
    plt.legend(title='Split')
    plt.grid(True)
    plt.tight_layout()
    # save the image file into results/plots
    name = ssd_tag.replace('/', '_').replace(' ', '_')
    plt.savefig(f"./results/plots/ssd_{name}.png")
    plt.close()

In [8]:
yolo_tags = data['yolo']['split_1'].Tags()['scalars']
yolo_tags

['train/box_loss',
 'train/cls_loss',
 'train/dfl_loss',
 'lr/pg0',
 'lr/pg1',
 'lr/pg2',
 'metrics/precision(B)',
 'metrics/recall(B)',
 'metrics/mAP50(B)',
 'metrics/mAP50-95(B)',
 'val/box_loss',
 'val/cls_loss',
 'val/dfl_loss']

In [9]:
custom_titles = {
    'metrics/precision(B)': "YOLO Precision",
    'metrics/recall(B)': "YOLO Recall",
    'metrics/mAP50(B)': "YOLO mAP",
}

for yolo_tag in yolo_tags:
    
    df = pd.DataFrame()
    #set a column for each split
    for split in data['yolo']:
        split_data_complete_run = []
        split_data = data['yolo'][split].Scalars(yolo_tag)
        for event_data in split_data:
            #split_data_complete_run[event_data.step] = event_data.value
            split_data_complete_run.append((event_data.step, event_data.value))
        
        # convert to dataframe
        split_data_complete_run = pd.DataFrame(split_data_complete_run, columns=['step', split])
        #set index to step
        split_data_complete_run.set_index('step', inplace=True)
        #merge with df
        if df.empty:
            df = split_data_complete_run
        else:
            df = df.join(split_data_complete_run, how='outer')

    #plot the data
    plt.figure(figsize=(10, 6)) 
    sns.lineplot(
        data=df,
        dashes=False,
        alpha=0.5
    )

    
    # create a mean column, it is the mean of all the splits
    mean = df.mean(axis=1)
    sns.lineplot(
        data=mean,
        label="Mean",
        linewidth=2,
        color="red"
    )

    plt.title(custom_titles[yolo_tag] if yolo_tag in custom_titles else yolo_tag)
    plt.xlabel('Step')
    plt.ylabel('Value')
    plt.legend(title='Split')
    plt.grid(True)
    plt.tight_layout()

    # save the image file into results/plots
    name = yolo_tag.replace('/', '_').replace(' ', '_')
    plt.savefig(f"./results/plots/yolo_{name}.png")
    plt.close()